# Naivni algoritam,  DLT algoritam i modifikovani DLT algoritam

In [1]:
import numpy as np
from numpy import linalg as LA

# Naivni algoritam

Moze da radi za najvise **4 tacke** i samim tim ima manju preciznost u odnosu na ostale. Ovim algoritmom nalazimo **matricu preslikavanja** tacaka. Funkcija **naivniAlgoritam** kao argumente prima originalne 4 tacke i 4 tacke u koje originalne tacke treba da se preslikaju koriscenjem dobijene matrice(tacke su date u **homogenim** koordinatama). Prvo trazimo koeficijente α, β i γ funkcijom **coefs** za oreginalne tacke, a zatim i za tacke u koje originalne zelimo da preslikamo zatim pravimo matricu P1 koja kao kolone ima αA, βB i γC gde su A, B i C oreginalne tacke, i tako isto P2 za tacke u koje originalne slikamo. Na kraju cemo konacnu matricu preslikavanja dobiti tako sto primenimo formulu P = P2 * inverz(P1).

In [2]:
def coefs(matrix):
    A = matrix.T[:,:3]
    B = matrix[3]
    A_inv = LA.inv(A)
    X = LA.inv(A).dot(B.T)
    return X

In [3]:
def naivniAlgoritam(original, slika):
    p1 = coefs(original)
    p2 = coefs(slika)
    
    P1 = np.array([[p1[i] * x for x in original[i]] for i in range(3)]).T
    P2 = np.array([[p2[i] * x for x in slika[i]] for i in range(3)]).T
    
    P = P2.dot(LA.inv(P1))
    return P

Matrica koju daje naivni algoritam

In [4]:
originali = np.array([[-3,-1,1], [3,-1,1], [1,1,1], [-1,1,1]])
slike = np.array([[-2,-1,1], [2,-1,1], [2,1,1], [-2,1,1]])
P = naivniAlgoritam(originali, slike)
P = np.round(P, decimals=5)
print('Naivni algoritam daje matricu projektivnog preslikavanja:\n', P)

Naivni algoritam daje matricu projektivnog preslikavanja:
 [[ 2.  0.  0.]
 [ 0.  2. -1.]
 [ 0. -1.  2.]]


# DLT algoritam

Moze da radi za vise od 4 tacke za razliku od naivnog pa mu je i preciznost veca. Prvo je potrebno da odredimo matricu korespodencije 2 X 9 za sve originalne tacke i tacke u koje preslikavamo oreginalne, zatim ih spajamo jednu na drugu i dobijamo matricu 2*N X 9 gde je N broj tacaka koje preslikavamo. Nakon toga vrsimo SVD  dekompoziciju matrice. Rezultujuca matrica ce biti poslednja kolona matrice V sto je u stvari poslednja vrsta matrice V_transponovano. Zatim rezultujucu matricu projektivnog preslikavanja skaliramo kako bismo poredili sa naivnim algoritmom(mnozimo sa 2 jer je kod matrice naivnog algoritma P[0][0] = 2).

In [5]:
def matricaKorespodencije(o, s):
    M = np.array([[0, 0, 0, -s[2]*o[0], -s[2]*o[1], -s[2]*o[2], s[1]*o[0], s[1]*o[1], s[1]*o[2]],
                 [s[2]*o[0], s[2]*o[1], s[2]*o[2], 0, 0, 0, -s[0]*o[0], -s[0]*o[1], -s[0]*o[2]]])
    return M

In [6]:
def dltAlgoritam(n, originali, slike):
    
    M1 = matricaKorespodencije(originali[0], slike[0])
    
    for i in range(1,n):
        M2 = matricaKorespodencije(originali[i], slike[i])
        M1 = np.vstack((M1, M2))
            
    U, D, V_transponovano = LA.svd(M1, full_matrices=True)
    
    P = V_transponovano[-1]
    P = P.reshape(3,3)
    return P

Vidimo da su matrice koje daju naivni algoritam i dlt algoritam iste nakon skaliranja.

In [7]:
originali = np.array([[-3,-1,1], [3,-1,1], [1,1,1], [-1,1,1], [1, 2, 3], [- 8, - 2, 1]])
slike = np.array([[-2,-1,1], [2,-1,1], [2,1,1], [-2,1,1], [2, 1, 4], [- 16, - 5, 4]])

P = dltAlgoritam(6, originali, slike)
P = np.round(P, decimals=5)
print("DLT algoritam daje matricu projektivnog preslikavanja:\n", P)

P = P/P[0][0]*2
print("Skalirana matrica:\n", P)

DLT algoritam daje matricu projektivnog preslikavanja:
 [[-0.53452  0.      -0.     ]
 [ 0.      -0.53452  0.26726]
 [-0.       0.26726 -0.53452]]
Skalirana matrica:
 [[ 2. -0.  0.]
 [-0.  2. -1.]
 [ 0. -1.  2.]]


# Modifikovani DLT algotitam(sa normalizacijom)

Takodje kao i DLT moze raditi sa vise tacaka i dosta je precizan. Modifikujemo ovaj algoritam kako bi radio preciznije i sa manjim brojevima jer cemo u praksi imati mnogo vise od npr. 6 tacaka.  **Normalizaciju** tacaka vrsimo tako sto prvo odredimo teziste tacaka , pravimo matricu koja translira teziste u koordinatni pocetak(G), svaku tacku transliramo za vektor TO(T je teziste, O koordinatni pocetak). Nakon toga racunamo prosecno rastojanje tacaka od koordinatnog pocetka, pravimo matricu homotetije kako bismo skalirali tacke da prosecka udaljenost bude koren(2), skaliramo tacke i na kraju matricu normalizacije dobijamo kao proizvod matrice homotetije i matrice translacije. Normalizovani DLT algoritam funkcionise tako sto prvo normalizujemo originalne tacke i tacke u koje se originalne slikaju, dobijamo pomocnu matricu tako sto primenimo klasican DLT na normalizovane tacke, a samu matricu projektivnog preslikavanja dobijamo kao proizvod inverza matrice normalizacije tacaka u koje se slikaju oreginalne tacke, pomocne matrice i matrice normalizacije oreginalnih tacaka.

In [8]:
def homogenizacija(tacka):
    return [tacka[0], tacka[1], 1]

In [9]:
def dehomogenizacija(tacka):
    return [tacka[0]/tacka[2], tacka[1]/tacka[2]]

In [10]:
def normalizacija(n,tacke):
    afine = []

    tx = 0
    ty = 0
    for i in range(n):
        afine.append(dehomogenizacija(tacke[i]))
        tx = tx + afine[i][0]
        ty = ty + afine[i][1]
    tx = tx/n
    ty = ty/n

    G = np.array([[1,0,-tx], [0,1,-ty], [0,0,1]])

    for i in range(n):
        afine[i] = G.dot(homogenizacija(afine[i]))
   
    dist = 0
    for i in range(n):
        dist = dist + np.sqrt(np.square(afine[i][0]) + np.square(afine[i][1]))

    dist = dist/n

    S = np.array([[np.sqrt(2)/dist,0, 0], [0,np.sqrt(2)/dist,0], [0,0,1]])

    for i in range(n):
        afine[i] = S.dot(homogenizacija(afine[i]))

    T = S.dot(G)
  
    return T, afine

In [11]:
def normDltAlgoritam(n, originali, slike):
    originali_t, originali_n = normalizacija(n, originali)

    slike_t, slike_n = normalizacija(n, slike)

    MP = dltAlgoritam(n, originali_n, slike_n)

    M = MP.dot(originali_t)
    M = LA.inv(slike_t).dot(M)

    return np.round(M, decimals=5)

Vidimo da su matrice koje daju naivni algoritam, dlt algoritam i modifikovani dlt algoritam iste nakon skaliranja.

In [12]:
originali = np.array([[-3,-1,1], [3,-1,1], [1,1,1], [-1,1,1], [1, 2, 3], [- 8, - 2, 1]])
slike = np.array([[-2,-1,1], [2,-1,1] ,[2,1,1], [-2,1,1] ,[2, 1, 4], [- 16, - 5, 4]])

P = normDltAlgoritam(6, originali, slike)
print("Normalizovani DLT algoritam daje matricu projektivnog preslikavanja:\n", P)

P = P/P[0][0]*2
print("Skalirana matrica:\n", P)

Normalizovani DLT algoritam daje matricu projektivnog preslikavanja:
 [[-0.397   0.     -0.    ]
 [ 0.     -0.397   0.1985]
 [-0.      0.1985 -0.397 ]]
Skalirana matrica:
 [[ 2. -0.  0.]
 [-0.  2. -1.]
 [ 0. -1.  2.]]


# Model za predaju domaceg

In [13]:
def pom(cc, originali):
    niz_tacaka = np.array(cc.dot(originali[0]))
    for i in range(1,7):
        T = cc.dot(originali[i])
        niz_tacaka = np.vstack((niz_tacaka, T))
    return niz_tacaka

In [14]:
P = np.array([[0, 3, 5], [4, 0, 0], [-1, -1, 6]])
print('Polazna matrica preslikavanja:\n', P)
print('\n')

originali = np.array([[-3, 2, 1], [-2, 5, 2], [1, 0, 3], [-7, 3, 1]])
slike = np.array([[11, -12, 7], [25, -8, 9], [15, 4, 17], [14, -28, 10]])

P = np.round(naivniAlgoritam(originali, slike), decimals=5)
print("Naivni algoritam daje matricu preslikavanja:\n", P)
print('\n')

originali = np.array([[-3, 2, 1], [-2, 5, 2], [1, 0, 3], [-7, 3, 1], [2, 1, 2], [-1, 2, 1], [1, 1, 1]])
slike = np.array([[11, -12, 7], [25, -8, 9], [15, 4, 17], [14, -28, 10], [13, 8, 9], [11, -4, 5], [8.02, 4, 4]])

P = dltAlgoritam(6, originali, slike)
print('DLT algoritam daje matricu preslikavanja:\n', P)

P = P/P[0][1]*3
print('Skalirana matrica:\n', P)

P = np.round(P, decimals=2)
print('Zaokruzena matrica:\n', P)
print('\n')

P = normDltAlgoritam(6, originali, slike)
print('Normalizovan DLT algoritam daje matricu preslikavanja:\n', P)

P = P/P[0][1]*3
print('Slakirana matrica:\n', P)

P = np.round(P, decimals=2)
print('Zaokruzena matrica:\n', P)
print('\n')

cc = np.array([[0,1,2], [-1, 0, 3], [0, 0, 1]])

O = pom(cc, originali)
print('Primena matrice cc na originale:\n', O)

S = pom(cc, slike)
print('Primena matrice cc na slike:\n', S)
print('\n')

print("dlt:\n")
P = dltAlgoritam(7, O, S)
print(P)
print('\n')
Ps = LA.inv(cc).dot(P).dot(cc)
print(Ps)
print('\n')
Pp = Ps/Ps[0][1]*3
print(Pp)

print("\nmodifikovani dlt:\n")
P = normDltAlgoritam(7, O, S)
print(P)
print('\n')
Ps = LA.inv(cc).dot(P).dot(cc)
print(Ps)
print('\n')
Pp = Ps/Ps[0][1]*3
print(Pp)

Polazna matrica preslikavanja:
 [[ 0  3  5]
 [ 4  0  0]
 [-1 -1  6]]


Naivni algoritam daje matricu preslikavanja:
 [[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]]


DLT algoritam daje matricu preslikavanja:
 [[-0.00000000e+00 -3.19801075e-01 -5.33001791e-01]
 [-4.26401433e-01 -3.45015596e-17 -4.82600207e-17]
 [ 1.06600358e-01  1.06600358e-01 -6.39602149e-01]]
Skalirana matrica:
 [[ 0.00000000e+00  3.00000000e+00  5.00000000e+00]
 [ 4.00000000e+00  3.23653318e-16  4.52719124e-16]
 [-1.00000000e+00 -1.00000000e+00  6.00000000e+00]]
Zaokruzena matrica:
 [[ 0.  3.  5.]
 [ 4.  0.  0.]
 [-1. -1.  6.]]


Normalizovan DLT algoritam daje matricu preslikavanja:
 [[ 0.      -0.22489 -0.37482]
 [-0.29985 -0.      -0.     ]
 [ 0.07496  0.07496 -0.44978]]
Slakirana matrica:
 [[-0.          3.          5.00004447]
 [ 3.99995553  0.          0.        ]
 [-0.99995553 -0.99995553  6.        ]]
Zaokruzena matrica:
 [[-0.  3.  5.]
 [ 4.  0.  0.]
 [-1. -1.  6.]]


Primena matrice cc na originale:
 [[ 4  6

# Provera domaceg

In [15]:
# 1)
print('1)')
originali_y = np.array([[2,1,1], [1,2,1], [3,4,1], [-1,-3,1]])
slike_y = np.array([[0,1,1], [5,0,1], [2,-5,1], [-1,-1,1]])

Pn = naivniAlgoritam(originali_y, slike_y)
Pn = Pn/Pn[0][0]  * 1.0

print('Naivni:\n', Pn)

Pd = dltAlgoritam(4, originali_y, slike_y)
Pd = Pd/Pd[0][0] * 1.0

print('Dlt:\n', Pd)

Pdn = normDltAlgoritam(4, originali_y, slike_y)
Pdn = Pdn/Pdn[0][0] * 1.0

print('Dlt norm:\n', Pdn)
print('\n')

# 2)
print('2)')

originali_y = np.array([[2,1,1], [1,2,1], [3,4,1], [-1,-3,1], [-2,5,1]])
slike_y = np.array([[0,1,1], [5,0,1], [2,-5,1], [-1,-1,1], [4,1,2]])

Pd = dltAlgoritam(5, originali_y, slike_y)
Pd = Pd/Pd[0][0] * 1.0

print('Dlt:\n', Pd)

Pdn = normDltAlgoritam(5, originali_y, slike_y)
Pdn = Pdn/Pdn[0][0] * 1.0

print('Dlt norm:\n', Pdn)
print('\n')

# 3)
print('3)')

originali_yn = np.array([[0,-3,1], [0,-1,1], [4,-1,1], [-7,-4,1], [0,5,1]])
slike_yn = np.array([[3,-1,1], [4,4,1], [9,1,1], [5,-2,1], [7,2,2]])

Pdn = normDltAlgoritam(5, originali_yn, slike_yn)
Pdn = Pdn/Pdn[0][0] * 1.0

print('Dlt norm:\n', Pdn)
print('\n')

1)
Naivni:
 [[ 1.         -0.4893617  -1.5106383 ]
 [ 0.62765957 -0.04255319 -0.54255319]
 [ 0.5        -0.46808511  0.13829787]]
Dlt:
 [[ 1.         -0.4893617  -1.5106383 ]
 [ 0.62765957 -0.04255319 -0.54255319]
 [ 0.5        -0.46808511  0.13829787]]
Dlt norm:
 [[ 1.         -0.48934973 -1.51063268]
 [ 0.62765377 -0.04254833 -0.54255712]
 [ 0.49999121 -0.46808436  0.13830405]]


2)
Dlt:
 [[ 1.         -0.50215564 -1.45779447]
 [ 0.51618777 -0.00399712 -0.44345389]
 [ 0.46632343 -0.43192449  0.10723129]]
Dlt norm:
 [[ 1.         -0.50218349 -1.45773658]
 [ 0.51826357 -0.00508863 -0.44493172]
 [ 0.46676659 -0.4322934   0.10741645]]


3)
Dlt norm:
 [[ 1.          8.1878513  14.71482102]
 [-1.23459225  1.60704073  7.8510995 ]
 [-0.09180554  2.39562173  3.94320087]]




Vidimo da su dobijene vrednosti za deo 1) i 2) vrlo slicne za sve algritme, ali postoje mala odstupanja, razlog je to sto je naivan algoritam manje precizan od dlt, a dlt je manje precizan od modifikovanog dlt, takodje jos jedan razlog je taj da sto vise tacaka prosledimo dlt i modifikovanom dlt algoritmu dobijamo vecu preciznost od njih.